In [1]:
import numpy as np 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torch.optim as optimizer
from collections import OrderedDict
from torch.optim.lr_scheduler import ReduceLROnPlateau

import random
import time
import math
import os
import copy

In [2]:
%run "C:\SBU-3\Jupyter-Research\RSCR\\2 Machine Learning\\4 CVAE Anar\\CVAE Model.ipynb"

In [ ]:
class LatentDataset(Dataset):
    def __init__(self, latent_dim=300):
        self.latent_dim = latent_dim
        paths = np.load('C:\SBU-3\Jupyter-Research\RSCR\Saved Data\\TEST\\path_test2.npy')
        mechs = np.load('C:\SBU-3\Jupyter-Research\RSCR\Saved Data\\TEST\\mec_test2.npy')
        paths = paths.reshape((paths.shape[0], paths.shape[1]*paths.shape[2]))
        
        if mechs.shape[1] == 16:
            mechs = mechs.reshape((mechs.shape[0], mechs.shape[1]))
        else:
            mechs = mechs.reshape((mechs.shape[0], mechs.shape[1]*mechs.shape[2]))
            
        self.data = np.hstack((paths, mechs))
        #random.shuffle(self.data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #path_mech = {'data':self.data[idx]}
        #return path_mech
        path_mech = self.data[idx]
        return path_mech[:self.latent_dim].astype(np.float32), path_mech[self.latent_dim:].astype(np.float32)


In [3]:
mechanism_size = 16
curve_size = 300
latent_dim = 2048


#model = cVAE
model = cVAE(mechanism_size=16, latent_dim=2048, curve_size=300)
loaded_state = torch.load('C:\SBU-3\Jupyter-Research\RSCR\Saved Data\RSCR Model Journal\cvae_beta10',map_location='cuda:0')
model.load_state_dict(loaded_state)

<All keys matched successfully>

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

path = torch.from_numpy(np.load('elbow_norm.npy')).unsqueeze(0)
curve = path.reshape((path.shape[0], path.shape[1]*path.shape[2])).float()
path = path.squeeze(0)
prediction = []

with torch.no_grad():
    curve1 = curve.repeat(100, 1).to(device)
    curve = model.encoder.curve_embedding(curve1)
    z = torch.randn([100, latent_dim]).to(device)
    mec_hat = model.decoder(z, curve)
    
    mec_hat = mec_hat.cpu().detach()
    curve1 = curve1.cpu().detach()
    out = np.hstack((curve1,mec_hat))
    for ele in out:
        new = ele
        prediction.append(new)


np.save('C:\SBU-3\Jupyter-Research\RSCR\Saved Data\RSCR Prediction Journal\elbow.npy', prediction)

In [ ]:
dataset = LatentDataset()
data_loader = DataLoader(dataset, batch_size=1) 

In [ ]:
model.eval()
prediction = []
with torch.no_grad():
    for batch_inc, data in enumerate(data_loader):
        curve = data[0]
        mec = data[1]
        
        curve2 = curve.repeat(30,1).to(device)
        curve3 = model.encoder.curve_embedding(curve2)
        z = torch.randn([30, latent_dim]).to(device)
        mec_hat = model.decoder(z, curve3)
        
        mec_hat = mec_hat.cpu().detach()
        curve2 = curve2.cpu().detach()
        out = np.hstack((curve2,mec_hat))
        for ele in out:
            new = ele
            prediction.append(new)
    #predict_mecs = predict_mecs.cpu().detach().numpy().squeeze()

In [ ]:
prediction=np.array(prediction)

In [ ]:
len(prediction[0])

In [ ]:
len(prediction)

In [ ]:
np.save('C:\SBU-3\Jupyter-Research\RSCR\Saved Data\RSCR Prediction Journal\cvae_beta25_pre_test', prediction)